In [141]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [142]:
df = pd.read_csv('../Data/Raw/ufo_raw.csv')

In [143]:
df.head(1)

,datetime,City,State,Shape,Duration,Summary,Posted,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2006-01-01 01:20:00,I-80 (unknown city proximity),NY,Formation,0 days 00:00:20.000000000,"Traveling from Brooklyn, NY to Groton, NY on I...",6/7/19,NaN,NaN,NaN,NaN


In [144]:
wdf = df.copy()
wdf.columns

Index(['datetime', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

In [145]:
wdf['datetype'] = pd.to_datetime(wdf['datetime']).dt.date
wdf.head()

,datetime,City,State,Shape,Duration,Summary,Posted,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,datetype
0,2006-01-01 01:20:00,I-80 (unknown city proximity),NY,Formation,0 days 00:00:20.000000000,"Traveling from Brooklyn, NY to Groton, NY on I...",6/7/19,NaN,NaN,NaN,NaN,2006-01-01
1,2006-01-01 23:00:00,San Antonio/Laredo (between),TX,Triangle,0 days 00:03:00.000000000,Three separate sightins which ocurred in rural...,6/9/09,NaN,NaN,NaN,NaN,2006-01-01
2,2006-01-02 00:00:00,Ann Arbor,MI,Changing,0 days 00:03:00.000000000,"Probable HOAX: 5 ufos, silent black objects, m...",10/30/06,NaN,NaN,NaN,NaN,2006-01-02
3,2006-01-02 00:00:00,Fredericksburg,VA,Light,0 days 00:02:00.000000000,clear sky looked like a star 10:00 am in the m...,2/1/07,NaN,NaN,NaN,NaN,2006-01-02
4,2006-01-04 07:45:00,Scotland,PA,Light,0 days 00:01:00.000000000,"Small, white, light in the northeast sky...dis...",2/1/07,NaN,NaN,NaN,NaN,2006-01-04


In [146]:
wdf['dttyp'] = pd.to_datetime(wdf['datetype'])

In [147]:
wdf['weeknum'] = wdf['dttyp'].dt.week

<ipython-input-147-b9d7638e36fd>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  wdf['weeknum'] = wdf['dttyp'].dt.week


In [148]:
wdf['weekofyear'] = wdf['dttyp'].dt.weekofyear

<ipython-input-148-6f1049e9a1ef>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  wdf['weekofyear'] = wdf['dttyp'].dt.weekofyear


In [149]:
wdf['year'] = wdf['dttyp'].dt.year

In [150]:
wdf['year_week'] = wdf['year'].astype(str) + "-" + wdf['weekofyear'].astype(str)

In [151]:
wdf = wdf.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'datetype'])

In [152]:
wdf = wdf.drop(columns=['datetime','weeknum'])

In [153]:
wdf.head()

,City,State,Shape,Duration,Summary,Posted,dttyp,weekofyear,year,year_week
0,I-80 (unknown city proximity),NY,Formation,0 days 00:00:20.000000000,"Traveling from Brooklyn, NY to Groton, NY on I...",6/7/19,2006-01-01,52,2006,2006-52
1,San Antonio/Laredo (between),TX,Triangle,0 days 00:03:00.000000000,Three separate sightins which ocurred in rural...,6/9/09,2006-01-01,52,2006,2006-52
2,Ann Arbor,MI,Changing,0 days 00:03:00.000000000,"Probable HOAX: 5 ufos, silent black objects, m...",10/30/06,2006-01-02,1,2006,2006-1
3,Fredericksburg,VA,Light,0 days 00:02:00.000000000,clear sky looked like a star 10:00 am in the m...,2/1/07,2006-01-02,1,2006,2006-1
4,Scotland,PA,Light,0 days 00:01:00.000000000,"Small, white, light in the northeast sky...dis...",2/1/07,2006-01-04,1,2006,2006-1


In [154]:
wdf['dttyp'].dtypes

dtype('<M8[ns]')

In [155]:
ufodata = pd.DataFrame(wdf['year_week'])

In [156]:
ufodata['date_of_sight'] = wdf['dttyp']

In [157]:
ufodata['year'] = wdf['year']
ufodata['month'] = wdf['dttyp'].dt.month
ufodata['week_of_year'] = wdf['weekofyear']
ufodata['state'] = wdf['State']
ufodata['city'] = wdf['City']
ufodata['sight_summary'] = wdf['Summary']


In [158]:
ufodata['ufo_shape'] = wdf['Shape']

In [159]:
ufodata = ufodata.reset_index()

In [160]:
dt = pd.date_range("20060102","20200906", freq='W')
datedf = pd.DataFrame(dt).rename(columns={0:'startdayweek'})

datedf['weeknum'] = datedf['startdayweek'].dt.isocalendar().week
datedf['month'] = datedf['startdayweek'].dt.month
datedf['year'] = datedf['startdayweek'].dt.year
datedf['week_id'] = datedf['year'].astype(str)+"-"+datedf['weeknum'].astype(str)

In [161]:
ufocount = ufodata.copy().drop(columns=['index','state','city','sight_summary','ufo_shape']).rename(columns={'year_week':'week_id'} ).sort_values('week_id').reset_index()
ufocount.head(1)

,index,week_id,date_of_sight,year,month,week_of_year
0,2,2006-1,2006-01-02,2006,1,1


In [162]:
datedf.head(1)

,startdayweek,weeknum,month,year,week_id
0,2006-01-08,1,1,2006,2006-1


In [163]:
ufo_cleaned_fin = pd.merge(left=datedf, right=ufocount, left_on='week_id', right_on='week_id')

In [164]:
ufo_cleaned_fin

,startdayweek,weeknum,month_x,year_x,week_id,index,date_of_sight,year_y,month_y,week_of_year
0,2006-01-08,1,1,2006,2006-1,2,2006-01-02,2006,1,1
1,2006-01-08,1,1,2006,2006-1,3,2006-01-02,2006,1,1
2,2006-01-08,1,1,2006,2006-1,4,2006-01-04,2006,1,1
3,2006-01-15,2,1,2006,2006-2,5,2006-01-12,2006,1,2
4,2006-01-15,2,1,2006,2006-2,6,2006-01-12,2006,1,2
...,...,...,...,...,...,...,...,...,...,...
71616,2020-09-06,36,9,2020,2020-36,70922,2020-08-31,2020,8,36
71617,2020-09-06,36,9,2020,2020-36,70923,2020-08-31,2020,8,36
71618,2020-09-06,36,9,2020,2020-36,70920,2020-08-31,2020,8,36
71619,2020-09-06,36,9,2020,2020-36,70925,2020-08-31,2020,8,36


In [165]:
wdfweek = ufo_cleaned_fin.copy().drop(columns=['weeknum','index','year_x','month_x'])
wdfweek.head()

,startdayweek,week_id,date_of_sight,year_y,month_y,week_of_year
0,2006-01-08,2006-1,2006-01-02,2006,1,1
1,2006-01-08,2006-1,2006-01-02,2006,1,1
2,2006-01-08,2006-1,2006-01-04,2006,1,1
3,2006-01-15,2006-2,2006-01-12,2006,1,2
4,2006-01-15,2006-2,2006-01-12,2006,1,2


In [166]:
# store counts per week_id in "ufosperweek" df
ufosperweek = pd.DataFrame(wdfweek['startdayweek'].value_counts()).reset_index().rename(columns={'index':'startdateweek','startdayweek': 'weekly_ufocount'})
# review
ufosperweek.head(1)

,startdateweek,weekly_ufocount
0,2014-07-06,470


In [167]:
maxalltime = ufosperweek.weekly_ufocount.max() 
onepct = maxalltime/100

In [168]:
ufosperweek['rel_val_alltime'] = ufosperweek['weekly_ufocount']/onepct

In [169]:
ufosperweek['year'] = ufosperweek['startdateweek'].dt.year
ufosperweek['month'] = ufosperweek['startdateweek'].dt.month
ufosperweek['weeknum'] = ufosperweek['startdateweek'].dt.isocalendar().week 
ufosperweek['week_id'] = ufosperweek['year'].astype(str) + "-" + ufosperweek['weeknum'].astype(str)

In [170]:
ufosperweek.head()

,startdateweek,weekly_ufocount,rel_val_alltime,year,month,weeknum,week_id
0,2014-07-06,470,100.000000,2014,7,27,2014-27
1,2015-11-08,406,86.382979,2015,11,45,2015-45
2,2020-04-19,403,85.744681,2020,4,16,2020-16
3,2013-07-07,399,84.893617,2013,7,27,2013-27
4,2012-07-08,339,72.127660,2012,7,27,2012-27


In [171]:
pd.to_csv('../Data/Datasets/cleandate_weekly_reval.csv')

AttributeError: module 'pandas' has no attribute 'to_csv'